(Merge entre l'extraction des noms des auteurs effectuée par michel, et la comparaison avec le reste des données)


In [5]:
import Levenshtein
import os.path

from tqdm import tqdm
from SPARQLWrapper import SPARQLWrapper, JSON
from statistics import mean

import sys
sys.path.append('../')

from Identification_couples_livres.extract_books_from_DB import *

In [21]:
# Fonctions utiles

# Pour enlever la qualificatif entre () dans un nom provenant de DBpedia
def traiter_nom(chaine):
    c = chaine.split(' (')
    return c[0]


# Recherche des auteurs dans DBpedia français

Une première approche pour trouver les auteurs dans DBpedia consiste à trouver ceux qui sont associés à une catégorie désignant un auteur:

* Écrivain québécois
* Poète québécois
* Dramaturge québécois
* ...

On note que les catégories forment une hiérarchie dominée par la catégorie "Écrivain québécois" 

On note aussi que plusieurs auteurs ont un URI, mais ne sont le sujet d'aucun triplet dans DBpedia. On peut les retrouver en recherchant ceux qui sont l'objet de la relation <http://dbpedia.org/ontology/wikiPageWikiLink> 
avec un prix littéraire comme sujet. Le problème ici est que cette relation est vague et que l'objet peut 
être bien autre chose qu'une personne.

In [22]:
# Recherche dans DBpedia français
sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setQuery("""
  PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  SELECT DISTINCT ?s ?label ?naissance
  WHERE {
    ?s dcterms:subject ?cat .
    ?cat skos:broader* <http://fr.dbpedia.org/resource/Catégorie:Écrivain_québécois> .
  
    ?s  rdfs:label ?label . 
    OPTIONAL { ?s <http://dbpedia.org/ontology/birthDate> ?naissance .}
    FILTER (lang(?label) = "fr")
  } 
""")

sparql.setReturnFormat(JSON)
results_dbpedia_fr = sparql.query().convert()

In [23]:
filtre = ["Liste d'écrivains québécois par ordre alphabétique",
           "Association des écrivaines et des écrivains québécois pour la jeunesse",
           "Liste de poètes québécois",
           "Liste d'auteurs québécois par ordre chronologique",
           "Liste d'auteurs québécois par ordre chronologique de décès"]


# On récupère les données extraites par la requête SPARQL
ecrivains_dbpedia_fr = set([(b['s']['value'],
                         traiter_nom(b['label']['value']), 
                         b['naissance']['value'][:4] if 'naissance' in b else 'X') 
                        for b in results_dbpedia_fr['results']['bindings']])


# On retire ceux qui ne correspondent pas à des écrivains
ecrivains_dbpedia_fr = [(uri,nom,naiss) for (uri,nom,naiss) in ecrivains_dbpedia_fr if nom not in filtre]

print(len(ecrivains_dbpedia_fr), 'écrivains du Québec trouvés dans DBpedia français')
      
file_writers = open("../Data/DBpedia/ecrivains_dbpedia_fr.txt","w")
for e in ecrivains_dbpedia_fr:
    file_writers.write(e[0] + " ; " + e[1] + '\n')
file_writers.close()



1259 écrivains du Québec trouvés dans DBpedia français


### Recherche des auteurs dans DBpedia anglais

Plusieurs auteurs se retrouvent de DBpedia anglais. Même, certains sont présents seulement dans DBpedia anglais.
Pour les retrouver, on pourrait chercher ceux qui sont liés à la catégorie Writers_from_Quebec, ou une de ses sous-catégories. Mais cela couvrerait trop large, puisqu'il y a des sous-catégories, comme "McGill University alumni", qui ne représentent pas nécessairement des écrivains. Il faut donc lister toutes les sous-catégories pertinentes. Si on essaie d'utiliser toute la hiérarchie de Writers_from_Quebec et filtrer par la suite les instances du sous-arrbre dominé par la catégorie Academics_in_Quebec, ça ne marche pas, à cause d'un problème de mémoire sur le serveur.

À noter que l'on retrouve plus systématiquement les dates de naissance dans DBpedia anglais.
    
    

In [24]:

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   SELECT DISTINCT ?s ?label ?naissance  
   WHERE {
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Quebec> }
     UNION
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Montreal> }
     UNION
     {?s dcterms:subject <http://dbpedia.org/resource/Category:Writers_from_Quebec_City>}

     ?s rdfs:label ?label . 
     OPTIONAL { ?s <http://dbpedia.org/ontology/birthDate> ?naissance .}
     
     FILTER (lang(?label) IN ("fr", "en"))
    } 
""")
sparql.setReturnFormat(JSON)
results_dbpedia_en = sparql.query().convert()



In [25]:
ecrivains_dbpedia_en = set([(b['s']['value'],
                             traiter_nom(b['label']['value']),
                             b['naissance']['value'][:4] if 'naissance' in b else 'X')
                            for b in results_dbpedia_en['results']['bindings']])

print(len(ecrivains_dbpedia_en), 'écrivains du Québec trouvés dans DBpedia anglais')



file_writers = open("../Data/DBpedia/ecrivains_dbpedia_en.txt","w")
for e in ecrivains_dbpedia_en:
    file_writers.write(e[0] + " ; " + e[1] + '\n')
file_writers.close()
    
    
    

784 écrivains du Québec trouvés dans DBpedia anglais


Deux écrivains sont dédoublés parce qu'ils ont deux dates de naissances différentes (erreurs dans DBpedia anglais)
* Victor Barbeau
* Angeline Hango

À noter qu'il y a deux écrivains qui ont deux homonymes dans DBpedia français et qui se trouvent aussi dans DBpedia anglais:
* Pierre Morency
* Louis Émond

# Comparaison avec les auteurs contenus dans les bases de données

In [8]:
if os.path.isfile('../Data/authors_ls.json'):
    with open('../Data/authors_ls.json', 'r') as input_file:
        author_ls = json.load(input_file, cls=BookJSONDecoder)
else:
    all_books = generate_all_books()
    author_ls, _ = generate_all_authors(all_books)

with open('../Data/DBpedia/ecrivains_dbpedia_fr.txt', 'r') as input_file:
    DBpedia_authors = []
    for line in input_file:
        DBpedia_authors.append(line.split(';')[-1])

# Couple dont on estime que les deux auteurs sont les mêmes
couple_writers = []
# Couple dont on estime que les deux auteurs sont différents mais à la limite d'être les mêmes
neg_couple_writers = []

# Compte les livres des auteurs par base de donnée (permet de voir si une base de donnée n'a aucun match)
data_base_stats = {}
# Compte les auteurs de wikidata qui ont été trouvé dans nos bases de donnée
found_at_least_once = 0
# Compte le nombre de couples
nb_couple = 0
# Compte le nombre de fois qu'un auteur de wikidata a été trouvé dans nos bases de donnée
author_found_freq = {}

for author_DBpedia in tqdm(DBpedia_authors):
    DB_pedia_author_found_count = 0
    for DB_author, DB_author_books in author_ls.items():
        normalised_DBpedia_title = normalize(author_DBpedia)
        dist_auteur = Levenshtein.distance(DB_author, normalised_DBpedia_title)
        if max(1, min(len(DB_author), len(normalised_DBpedia_title)) / 6) > dist_auteur > max(1, min(len(author_DB), len(normalised_DBpedia_title)) / 8):
            neg_couple_writers.append({
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_DBpedia': author_DBpedia
                    })
        elif dist_auteur < max(1, min(len(DB_author), len(normalised_DBpedia_title)) / 8):
            new_couple = {
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_DBpedia': author_DBpedia
                    }
            nb_couple += 1
            DB_pedia_author_found_count += 1

            couple_writers.append(new_couple)
            for book in DB_author_books:
                try:
                    data_base_stats[book.data_base] += 1
                except:
                    data_base_stats[book.data_base] = 1

    if DB_pedia_author_found_count:
        found_at_least_once += 1
        author_found_freq[author_DBpedia] = DB_pedia_author_found_count


print("nombre d'autheurs en commun: ", found_at_least_once, " soit ", found_at_least_once * 100 / len(list(author_ls)), "%")
print("nombre d'autheurs reconnus: ", nb_couple, " soit ", nb_couple/found_at_least_once, ' auteur dans nos bases de donnée par auteur reconnu dans Dbpedia')
print("Frequence de reconnaissance des 10 auteurs de DBpedia les plus reconnus: ", sorted(author_found_freq.items(), key=lambda item: item[1], reverse=True)[:10])


print("\nRépartition des livres des auteurs reconnus dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats))

print("\n20 premiers couples à la limite d'être considérés comme égaux")
for couple in neg_couple_writers[:20]:
    print("Autheur DB: ", couple['author_DB']['name'] + " " * (40 - len(couple['author_DB']['name'])),
          "Autheur DBpedia: ", couple['author_DBpedia'])

with open('../Data/DBpedia/couple_authors_DBpedia.json', 'w') as outfile:
    json.dump(couple_writers, outfile, cls=BookJSONEncoder)

100%|██████████| 1259/1259 [03:17<00:00,  6.37it/s]


nombre d'autheurs en commun:  1005  soit  5.035574706884457 %
nombre d'autheurs reconnus:  1080  soit  1.0746268656716418  auteur dans nos bases de donnée par auteur reconnu dans Dbpedia
Frequence de reconnaissance des 10 auteurs de DBpedia les plus reconnus:  [(' Michel Tremblay\n', 3), (' Sylvie Bérard\n', 3), (' Denise Boucher\n', 3), (' Jacques Brossard\n', 3), (' Jocelyne Robert\n', 2), (' Jacques Godbout\n', 2), (' Michel David\n', 2), (' Paul Chanel Malenfant\n', 2), (' Louis Hamelin\n', 2), (' Christiane Lahaie\n', 2)]

Répartition des livres des auteurs reconnus dans les bases de données: 
1/8:  {"ILE": 16471, "Depot_legal": 9317, "ADP": 3184, "Babelio": 2568}

20 premiers couples à la limite d'être considérés comme égaux
Autheur DB:  chantal fortin                           Autheur DBpedia:   Chantal Morin

Autheur DB:  marco belanger                           Autheur DBpedia:   Marcel Bélanger

Autheur DB:  sylvain houde                            Autheur DBpedia:   Sylvain 

# Extraction  des champs disponibles depuis DBpédia pour les auteurs en commun

## Récupération de tout les champs

In [9]:
with open('../Data/DBpedia/ecrivains_dbpedia_fr.txt', 'r') as input_file:
    DBpedia_authors_uri = []
    for line in input_file:
        DBpedia_authors_uri.append(line.split(';')[0])

sparql = SPARQLWrapper("http://fr.dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

result = {}

for author_uri in DBpedia_authors_uri:
    time.sleep(1)
    sparql.setQuery("""
    SELECT DISTINCT ?hasprop ?v
    where {{ {0} ?hasprop ?v}}
""".format("<" + author_uri.replace(" ", "") + ">"))
    results_dbpedia_fr = sparql.query().convert()
    result[author_uri.replace(" ", "")] = [(duo['hasprop']['value'], duo['v']['value']) for duo in results_dbpedia_fr['results']['bindings']]

print('Exemple de champs: ', result[DBpedia_authors_uri[0]][0:2])

KeyError: 'http://fr.dbpedia.org/resource/Renaud_Longchamps '

## Tri et sauvegarde des champs

In [ ]:
prop_dict = {}
prop_counter = {}
for author_uri, properties in result.items():
    for property in properties:
        try:
            prop_counter[property[0]] += 1
        except:
            prop_counter[property[0]] = 1

for author_uri, properties in result.items():
    for rep_prop in prop_counter:
        counter = 0
        for property in properties:
            if property[0] == rep_prop:
                counter += 1
        try:
            prop_dict[rep_prop].append(counter)
        except:
            prop_dict[rep_prop] = [counter]

prop_resume = {}
for rep_prop in prop_dict:
    prop_resume[rep_prop] = {
        'non_null_mean': mean([e for e in prop_dict[rep_prop] if e]),
        'nb_non_null': sum([1 for e in prop_dict[rep_prop] if e]),
        'sum': sum(prop_dict[rep_prop]),
    }

print('Exemple: ', json.dumps(sorted(prop_resume.items(), key=lambda item: item[1]['nb_non_null'], reverse=True)[0:5], indent=1))

with open('../Data/DBpedia/list_properies_by_author.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['name', 'non_null_mean', 'nb_non_null', 'sum' ])
        writer.writeheader()
        for rep_prop_name, rep_prop_dict in prop_resume.items():
            rep_prop_dict['name'] = rep_prop_name
            writer.writerow(rep_prop_dict)